#Task 3: Language model using the text from a large collection of Science Fiction stories

In [4]:
#importing the necessary libraries
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Embedding, GRU, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file, to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
import random
import sys
import io
import requests
import re
import string
from google.colab import drive
import zipfile

In [5]:
#mounting google drive
drive.mount('/content/drive', force_remount=True) 

Mounted at /content/drive


Data preparation

In [6]:
#loading the data
zip_file = "/content/drive/My Drive/Deep Learning/SciFi.zip"
# extract location
extract_path = "/content/DeepLearning/Scifi/"
# create a ZipFile object
with zipfile.ZipFile(zip_file, 'r') as zip_obj:
  # extract all the contents of the zip file to the specified directory 
  zip_obj.extractall(extract_path)

In [7]:
#open the extracted zip file
file=open('/content/DeepLearning/Scifi/internet_archive_scifi_v3.txt','r',encoding='utf8').read()

In [8]:
len(file)

149326361

In [9]:
print(file[:500])

MARCH # All Stories New and Complete Publisher Editor IF is published bi-monthly by Quinn Publishing Company, Inc., Kingston, New York. Volume #, No. #. Copyright # by Quinn Publishing Company, Inc. Application for Entry' as Second Class matter at Post Office, Buffalo, New York, pending. Subscription # for # issues in U.S. and Possessions: Canada # for # issues; elsewhere #. Aiiow four weeks for change of address. All stories appearing in this magazine are fiction. Any similarity to actual perso


Data Preprocessing

In [10]:
#Data cleaning - Splitting the text, Removing the special characters and extra space
def cleaned_text(text):
  text = text.split(" ")
  text = re.sub(r'[^\x00-\x7f]', r'', str(text)) # removing special chars...
  text = text.translate(str.maketrans('', '', string.punctuation)) # removing special chars...
  text = re.sub('\s+', ' ', str(text)) # removing extra spaces...
  return text

text = cleaned_text(file)

In [30]:
#visualize cleaned text
print("Example of cleaned text:",text[0:100])
print("Length of text:", len(text))

Example of cleaned text: MARCH All Stories New and Complete Publisher Editor IF is published bimonthly by Quinn Publishing Co
Length of text: 142442755


In [27]:
#creating a corpus for the text and creating a list of unique words
text_corpus = text.split(" ")
text_corpus = [x for x in text_corpus if x != ""]
print("Length of text corpus:",len(text_corpus))
unique_vocab = list(set(text_corpus))
print("Length of unique words:",len(unique_vocab))

Length of text corpus: 26308635
Length of unique words: 330125


In [16]:
text_corpus[:20]

['MARCH',
 'All',
 'Stories',
 'New',
 'and',
 'Complete',
 'Publisher',
 'Editor',
 'IF',
 'is',
 'published',
 'bimonthly',
 'by',
 'Quinn',
 'Publishing',
 'Company',
 'Inc',
 'Kingston',
 'New',
 'York']

In [26]:
#Dividing the whole text into multiple strings each of length 31
seq_length =  30
step_size = 1 
all_sentences = []
for i in range(seq_length, len(text_corpus)):
  # sliding window to divide the text
  sentence = text_corpus[i - seq_length: i] 
  sentence = ' '.join(sentence)
  all_sentences.append(sentence)

all_sentences[:10]
print("Length of all sentences: ", len(all_sentences))

Length of all sentences:  499970


In [24]:
# Data selected and unique vocabulary in the data
model_sent  = all_sentences[25808605:]
print(len(model_sent), "words are selected from the end as the smaller dataset")
print("Example of model sent: ", model_sent[:20])
unique_vocab = list(set(model_sent))
print("Length of unique words: ", len(unique_vocab))
print("Example of of unique words: ", unique_vocab[0:10])

500000 words are selected from the end as the smaller dataset
Example of model sent:  ['think He fell forward Or more he just crumpled to a heap on his Formula Hut floor and became right before her eyes a very sad pile of quite expensive', 'He fell forward Or more he just crumpled to a heap on his Formula Hut floor and became right before her eyes a very sad pile of quite expensive replacement', 'fell forward Or more he just crumpled to a heap on his Formula Hut floor and became right before her eyes a very sad pile of quite expensive replacement metal', 'forward Or more he just crumpled to a heap on his Formula Hut floor and became right before her eyes a very sad pile of quite expensive replacement metal and', 'Or more he just crumpled to a heap on his Formula Hut floor and became right before her eyes a very sad pile of quite expensive replacement metal and fleshstrip', 'more he just crumpled to a heap on his Formula Hut floor and became right before her eyes a very sad pile of quit

In [25]:
# tokenizing the words and converting to numerical values
tokenizer = Tokenizer(num_words= 10000)
tokenizer.fit_on_texts(model_sent)
seq = tokenizer.texts_to_sequences(model_sent)

In [32]:
# Pad sequences to a fixed length of 30 with 'pre'
max_length = 30
padded_sequences = pad_sequences(seq, maxlen=max_length, padding='pre', truncating='pre')

In [33]:
data = np.vstack(padded_sequences)

In [34]:
# selecting 30 columns of each rows as features and 31st as target variable
X = data[:, :-1]
y = data[:, -1]

In [35]:
len(tokenizer.word_index)

21544

In [ ]:
X.shape[1]

29

Model Building

In [36]:
# Sequential LSTM model to predict next word...
model = Sequential()

# input_dim is the length of the vocab/dictionary that we created earlier, output_dim is 50, and input length is 31...
model.add(Embedding(len(tokenizer.word_index) + 1, 50, input_length = X.shape[1])) 

# 64 LSTM units and return_sequences = True to pass it on to next LSTM layer. Dropout for regularization
model.add(LSTM(64, return_sequences=True))


model.add(LSTM(64))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 29, 50)            1077250   
                                                                 
 lstm (LSTM)                 (None, 29, 64)            29440     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 21545)             2779305   
                                                                 
Total params: 3,927,339
Trainable params: 3,927,339
No

In [ ]:
#define early stopping and Model checkpoints that are used as callbacks in the model
es = EarlyStopping(monitor="loss",patience=10,verbose=0,mode="auto")
checkpoint_filepath = "/content/drive/MyDrive/Deep Learning/task3/checkpoints/task3_checkpoints.{epoch:007}"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath, monitor='loss', save_best_only=True, mode='min', verbose=1)


In [ ]:
# Compiling the model with adam optimizer and training it for 100 epochs with a batch size of 32
model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X, y, batch_size = 32, epochs=100, callbacks=[es,model_checkpoint_callback])
model.save('/content/drive/MyDrive/Deep Learning/task3/model_weights.h5')

Epoch 1/100
15625/15625 [==============================] - ETA: 0s - loss: 6.5371 - accuracy: 0.0819
Epoch 1: loss improved from inf to 6.53713, saving model to /content/drive/MyDrive/Deep Learning/task3/checkpoints/task3_checkpoints.0000001


15625/15625 [==============================] - 160s 10ms/step - loss: 6.5371 - accuracy: 0.0819
Epoch 2/100
15622/15625 [============================>.] - ETA: 0s - loss: 6.0613 - accuracy: 0.1057
Epoch 2: loss improved from 6.53713 to 6.06130, saving model to /content/drive/MyDrive/Deep Learning/task3/checkpoints/task3_checkpoints.0000002


15625/15625 [==============================] - 135s 9ms/step - loss: 6.0613 - accuracy: 0.1057
Epoch 3/100
15623/15625 [============================>.] - ETA: 0s - loss: 5.8554 - accuracy: 0.1155
Epoch 3: loss improved from 6.06130 to 5.85536, saving model to /content/drive/MyDrive/Deep Learning/task3/checkpoints/task3_checkpoints.0000003


15625/15625 [==============================] - 134s 9ms/step - loss: 5.8554 - accuracy: 0.1155
Epoch 4/100
15625/15625 [==============================] - ETA: 0s - loss: 5.7063 - accuracy: 0.1226
Epoch 4: loss improved from 5.85536 to 5.70628, saving model to /content/drive/MyDrive/Deep Learning/task3/checkpoints/task3_checkpoints.0000004


15625/15625 [==============================] - 135s 9ms/step - loss: 5.7063 - accuracy: 0.1226
Epoch 5/100
15625/15625 [==============================] - ETA: 0s - loss: 5.5919 - accuracy: 0.1284
Epoch 5: loss improved from 5.70628 to 5.59190, saving model to /content/drive/MyDrive/Deep Learning/task3/checkpoints/task3_checkpoints.0000005


...

15625/15625 [==============================] - 133s 8ms/step - loss: 4.2151 - accuracy: 0.2241
Epoch 94/100
15623/15625 [============================>.] - ETA: 0s - loss: 4.2139 - accuracy: 0.2246
Epoch 94: loss improved from 4.21508 to 4.21388, saving model to /content/drive/MyDrive/Deep Learning/task3/checkpoints/task3_checkpoints.0000094


15625/15625 [==============================] - 135s 9ms/step - loss: 4.2139 - accuracy: 0.2246
Epoch 95/100
15623/15625 [============================>.] - ETA: 0s - loss: 4.2190 - accuracy: 0.2244
Epoch 95: loss did not improve from 4.21388
15625/15625 [==============================] - 123s 8ms/step - loss: 4.2190 - accuracy: 0.2244
Epoch 96/100
15622/15625 [============================>.] - ETA: 0s - loss: 4.2141 - accuracy: 0.2244
Epoch 96: loss did not improve from 4.21388
15625/15625 [==============================] - 122s 8ms/step - loss: 4.2141 - accuracy: 0.2244
Epoch 97/100
15619/15625 [============================>.] - ETA: 0s - loss: 4.2123 - accuracy: 0.2244
Epoch 97: loss improved from 4.21388 to 4.21238, saving model to /content/drive/MyDrive/Deep Learning/task3/checkpoints/task3_checkpoints.0000097


15625/15625 [==============================] - 135s 9ms/step - loss: 4.2124 - accuracy: 0.2244
Epoch 98/100
15621/15625 [============================>.] - ETA: 0s - loss: 4.2124 - accuracy: 0.2251
Epoch 98: loss did not improve from 4.21238
15625/15625 [==============================] - 123s 8ms/step - loss: 4.2124 - accuracy: 0.2251
Epoch 99/100
15620/15625 [============================>.] - ETA: 0s - loss: 4.2085 - accuracy: 0.2255
Epoch 99: loss improved from 4.21238 to 4.20856, saving model to /content/drive/MyDrive/Deep Learning/task3/checkpoints/task3_checkpoints.0000099


15625/15625 [==============================] - 133s 9ms/step - loss: 4.2086 - accuracy: 0.2255
Epoch 100/100
15620/15625 [============================>.] - ETA: 0s - loss: 4.2060 - accuracy: 0.2254
Epoch 100: loss improved from 4.20856 to 4.20606, saving model to /content/drive/MyDrive/Deep Learning/task3/checkpoints/task3_checkpoints.0000100


15625/15625 [==============================] - 133s 8ms/step - loss: 4.2061 - accuracy: 0.2254


In [38]:
#model loaded from the previous training. Training is resumed 
model = load_model('/content/drive/MyDrive/Deep Learning/task3/model_weights.h5')
# Load the checkpoints
weight_file = "/content/drive/MyDrive/Deep Learning/task3/checkpoints/task3_checkpoints.0000101"
model.load_weights(weight_file).expect_partial()

In [ ]:
#Resuming the model training by loading the previous weights
model.fit(X,y, epochs=150, batch_size = 32 ,initial_epoch=101, callbacks=[es, model_checkpoint_callback])

Epoch 102/150
15624/15625 [============================>.] - ETA: 0s - loss: 4.2095 - accuracy: 0.2252
Epoch 102: loss improved from inf to 4.20946, saving model to /content/drive/MyDrive/Deep Learning/task3/checkpoints/task3_checkpoints.0000102


15625/15625 [==============================] - 156s 9ms/step - loss: 4.2095 - accuracy: 0.2252
Epoch 103/150
15624/15625 [============================>.] - ETA: 0s - loss: 4.2042 - accuracy: 0.2259
Epoch 103: loss improved from 4.20946 to 4.20423, saving model to /content/drive/MyDrive/Deep Learning/task3/checkpoints/task3_checkpoints.0000103


15625/15625 [==============================] - 133s 8ms/step - loss: 4.2042 - accuracy: 0.2259
Epoch 104/150
15621/15625 [============================>.] - ETA: 0s - loss: 4.2066 - accuracy: 0.2255
Epoch 104: loss did not improve from 4.20423
15625/15625 [==============================] - 122s 8ms/step - loss: 4.2067 - accuracy: 0.2255
Epoch 105/150
15625/15625 [==============================] - ETA: 0s - loss: 4.2069 - accuracy: 0.2259
Epoch 105: loss did not improve from 4.20423
15625/15625 [==============================] - 122s 8ms/step - loss: 4.2069 - accuracy: 0.2259
Epoch 106/150
15619/15625 [============================>.] - ETA: 0s - loss: 4.2102 - accuracy: 0.2254
Epoch 106: loss did not improve from 4.20423
15625/15625 [==============================] - 122s 8ms/step - loss: 4.2102 - accuracy: 0.2254
Epoch 107/150
15621/15625 [============================>.] - ETA: 0s - loss: 4.2082 - accuracy: 0.2262
Epoch 107: loss did not improve from 4.20423
15625/15625 [===============

15625/15625 [==============================] - 130s 8ms/step - loss: 4.2034 - accuracy: 0.2259
Epoch 109/150
15625/15625 [==============================] - ETA: 0s - loss: 4.2037 - accuracy: 0.2264
Epoch 109: loss did not improve from 4.20342
15625/15625 [==============================] - 122s 8ms/step - loss: 4.2037 - accuracy: 0.2264
Epoch 110/150
15623/15625 [============================>.] - ETA: 0s - loss: 4.1984 - accuracy: 0.2259
Epoch 110: loss improved from 4.20342 to 4.19850, saving model to /content/drive/MyDrive/Deep Learning/task3/checkpoints/task3_checkpoints.0000110


15625/15625 [==============================] - 133s 8ms/step - loss: 4.1985 - accuracy: 0.2259
Epoch 111/150
15620/15625 [============================>.] - ETA: 0s - loss: 4.2007 - accuracy: 0.2269
Epoch 111: loss did not improve from 4.19850
15625/15625 [==============================] - 121s 8ms/step - loss: 4.2007 - accuracy: 0.2269
Epoch 112/150
15622/15625 [============================>.] - ETA: 0s - loss: 4.1994 - accuracy: 0.2265
Epoch 112: loss did not improve from 4.19850
15625/15625 [==============================] - 121s 8ms/step - loss: 4.1993 - accuracy: 0.2265
Epoch 113/150
15620/15625 [============================>.] - ETA: 0s - loss: 4.2017 - accuracy: 0.2265
Epoch 113: loss did not improve from 4.19850
15625/15625 [==============================] - 121s 8ms/step - loss: 4.2016 - accuracy: 0.2265
Epoch 114/150
15619/15625 [============================>.] - ETA: 0s - loss: 4.2016 - accuracy: 0.2266
Epoch 114: loss did not improve from 4.19850
15625/15625 [===============

The loss did not imrpove after 110 epochs where the training is terminated as the patience set in early stopping is set to 0

In [47]:
#model loaded from the previous training. Training is resumed 
model = load_model('/content/drive/MyDrive/Deep Learning/task3/model_weights.h5')
# Load the checkpoints
weight_file = "/content/drive/MyDrive/Deep Learning/task3/checkpoints/task3_checkpoints.0000110"
model.load_weights(weight_file).expect_partial()

In [40]:
# function to generate text when input text and number of words to be generated are given. The text is preprocessed. 
def text_generator(model, tokenizer, seq_len, feature_text, num_words):
  text = []
  for i in range(num_words):
    token = tokenizer.texts_to_sequences([feature_text])[0]
    token = pad_sequences([token], maxlen = seq_len, truncating='pre')
    # y_pred = model.predict_classes(token)
    y_pred = model.predict(token) 
    y_pred = np.argmax(y_pred, axis=1)

    pred_word = ''
    for word, idx in tokenizer.word_index.items():
      if idx == y_pred:
        pred_word = word
        break
    feature_text += " "+ pred_word
    text.append(pred_word)

  return " ".join(text)

In [46]:
while True:
    text = input('Enter your line: ')
    if text.lower() == 'q':
        print('Session terminated...')
        break

    num_of_words = int(input('Enter the number of words to generate: '))

    try:
        print(text)

        next_sentences = text_generator(model, tokenizer, X.shape[1], text, num_of_words)
        print("The next sentences are:", next_sentences)
        print("Enter q to quit")

    except Exception as e:
        print('Error occurred:', e)
        continue


Enter your line: sea monsters are coming to get us and we have to get back to the shore as soon as possible
Enter the number of words to generate: 50
sea monsters are coming to get us and we have to get back to the shore as soon as possible
1/1 [==============================] - 0s 36ms/step
The next sentences are: astronomers to allow nine down in the sun whose tongue is written in the cells the captain is the same thing the chief scientist came together marin had grown forever weapons is hardly the final landing the golden section says appeared for the extreme stage and successful supply in the
Enter q to quit
Enter your line: q
Session terminated...
